In [1]:
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib import style
import numpy as np
import pickle
import os
from sklearn.decomposition import NMF, PCA
from sklearn.cluster import KMeans
from importlib import reload
from trecs.metrics import MSEMeasurement, InteractionSpread, InteractionSpread, InteractionSimilarity, RecSimilarity, RMSEMeasurement, InteractionMeasurement
from trecs.components import Users
import trecs.matrix_ops as mo
import src.globals as globals
import seaborn as sns

from wrapper.models.bubble import BubbleBurster
from src.utils import get_topic_clusters, create_embeddings, load_or_create_measurements_df, load_and_process_movielens, collect_parameters, load_measurements
from src.scoring_functions import cosine_sim, entropy, content_fairness
from wrapper.metrics.evaluation_metrics import SerendipityMetric, DiversityMetric, NoveltyMetric, RecallMeasurement

random_state = np.random.seed(42)
plt.style.use("seaborn")

# import warnings filter
from warnings import simplefilter
# ignore all future warnings
simplefilter(action='ignore', category=FutureWarning)

globals.initialize()

## Creating RecommenderSystem on MovieLens 

In [2]:
n_attrs=20
max_iter=1000
n_clusters=20

In [3]:
binary_ratings_matrix = load_and_process_movielens(file_path='data/ml-100k/u.data')

In [4]:
# Get user and item representations using NMF
user_representation, item_representation = create_embeddings(binary_ratings_matrix, n_attrs=n_attrs, max_iter=max_iter)

Loaded embeddings.


In [5]:
# Define topic clusters using NMF
item_topics = get_topic_clusters(binary_ratings_matrix, n_clusters=n_clusters, n_attrs=n_attrs, max_iter=max_iter)

Loaded clusters.


In [6]:
# Create user clusters based off how often they interacted with the same item
print('Calculating clusters...')
co_occurence_matrix = binary_ratings_matrix @ binary_ratings_matrix.T
print(co_occurence_matrix.shape)

# Matrix factorize co_occurence_matrix to get embeddings
nmf_cooc = NMF(n_components=n_attrs, max_iter=max_iter)
W_topics = nmf_cooc.fit_transform(co_occurence_matrix)

# cluster W_topics
cluster_ids = KMeans(n_clusters=n_clusters, max_iter=max_iter, random_state=random_state).fit_predict(W_topics)

print('Calculated clusters.')

Calculating clusters...
(943, 943)


/opt/conda/envs/rsenv2/lib/python3.7/site-packages/sklearn/decomposition/_nmf.py:1641: ConvergenceWarning: Maximum number of iterations 1000 reached. Increase it to improve convergence.
  ConvergenceWarning,


Calculated clusters.


# Simulation

### Model

In [7]:
score_fn = ''
probabilistic = False
globals.ALPHA = 0.2
alpha = globals.ALPHA

# User parameters
drift = 0.2
attention_exp=1.5
pair_all=False

In [8]:
num_users = len(user_representation)
num_items = len(item_representation)
print(f'Number of items: {num_items}')
print(f'Number of users: {num_users}')

if pair_all:
    # All possible user pairs
    user_pairs = [(u_idx, v_idx) for u_idx in range(len(user_representation)) for v_idx in range(len(user_representation))]

else:
    # create user_pairs by pairing users only with others that are not in the same cluster
    user_pairs = []
    for u_idx in range(num_users):
        for v_idx in range(num_users):
            if cluster_ids[u_idx] != cluster_ids[v_idx]:
                user_pairs.append((u_idx, v_idx))
print(f'Number of user pairs: {len(user_pairs)}')
                
users = Users(actual_user_profiles=user_representation, 
              repeat_interactions=False, 
              drift=drift,
              attention_exp=attention_exp)

Number of items: 20
Number of users: 943
Number of user pairs: 790474


In [9]:
mse = MSEMeasurement()
measurements_list = [
    InteractionMeasurement(), 
    mse,  
    InteractionSpread(), 
    InteractionSimilarity(pairs=user_pairs), 
    RecSimilarity(pairs=user_pairs), 
    SerendipityMetric(), 
    DiversityMetric(), 
    NoveltyMetric(),
    RecallMeasurement(),
    # MeanNumberOfTopics(),
]

In [10]:
# Model
config = {
    'actual_user_representation': users,
    'actual_item_representation': item_representation,
    'item_topics': item_topics,
    'num_attributes': n_attrs,
    'num_items_per_iter': 10,
    'seed': 42,
    'record_base_state': True,
}

model_name='myopic'
requires_alpha = False

if score_fn:
    if score_fn == 'cosine_sim':
        config['score_fn'] = cosine_sim
        requires_alpha = True
    elif score_fn == 'entropy':
        config['score_fn'] = entropy
        requires_alpha = True
    elif score_fn == 'content_fairness':
        config['score_fn'] = content_fairness        
    else:
        raise Exception('Given score function does not exist.')
    model_name = score_fn

if probabilistic:
    config['probabilistic_recommendations'] = True
    model_name += '_prob'

In [11]:
model = BubbleBurster(**config)

model.add_metrics(*measurements_list)

print("Model representation of users and items are given by:")
print(f"- An all-zeros matrix of users of dimension {model.predicted_user_profiles.shape}")
print(f"- A randomly generated matrix of items of dimension {model.predicted_item_attributes.shape}")

Model representation of users and items are given by:
- An all-zeros matrix of users of dimension (943, 20)
- A randomly generated matrix of items of dimension (20, 1682)


In [12]:
# Fair Model
train_timesteps=5
model.startup_and_train(timesteps=train_timesteps)

100%|█████████████████████████████████████████████████████████████████████████████████████| 5/5 [00:37<00:00,  7.55s/it]


In [13]:
run_timesteps=20
model.run(timesteps=run_timesteps)

  0%|                                                                                            | 0/20 [00:09<?, ?it/s]


NameError: name 'mo' is not defined

In [ ]:
import src
reload(src.utils)
from src.utils import *
    
# Save measurements
measurements_dir = f'artefacts/measurements/'
file_name = f'{model_name}_measurements_{train_timesteps}trainTimesteps_{run_timesteps}runTimesteps_{n_attrs}nAttrs_{n_clusters}nClusters_{drift}Drift_{attention_exp}AttentionExp_{pair_all}PairAll'
measurements_path = measurements_dir + file_name
if requires_alpha:
    measurements_path += f'_{alpha}Lambda'
measurements_path += '.csv'
measurements_df = load_or_create_measurements_df(model, model_name, train_timesteps, measurements_path)
measurements_df.to_csv(measurements_path)
print('Measurements saved.')

# Create df for parametrics
numeric_cols = ['trainTimesteps', 'runTimesteps', 'nAttrs', 'nClusters', 'Lambda']
columns = ['model_name'] + numeric_cols

data = [[model_name, train_timesteps, run_timesteps, n_attrs, n_clusters, None]]
if requires_alpha:
    data = [[model_name, train_timesteps, run_timesteps, n_attrs, n_clusters, alpha]]

parameters_df = pd.DataFrame(data,
                             columns = columns)
for col in numeric_cols:
    parameters_df[col] = pd.to_numeric(parameters_df[col])
    
plot_measurements([measurements_df], parameters_df)

# Experiments

In [ ]:
recommender = model

In [ ]:
# Novelty that is based on probability calculation from generate_recommendations 
# (leave previously interacted items out)
item_counts = recommender.item_count
item_counts[item_counts == 0] = 1
items_self_info = (-1) * np.log(item_counts)

# turn scores in probability distribution over items to ensure that all independent of the ranking function, the metric yields comparable values
scores = recommender.predicted_scores.value
probabilities = scores / np.sum(scores, axis=1)[:, np.newaxis]

item_indices = recommender.indices
if not recommender.users.repeat_interactions:
    # for each user, eliminate items that have been interacted with
    item_indices = item_indices[np.where(item_indices >= 0)]
    item_indices = item_indices.reshape((recommender.num_users, -1))

s_filtered = mo.to_dense(recommender.predicted_scores.filter_by_index(item_indices))
row = np.repeat(recommender.users.user_vector, item_indices.shape[1])
row = row.reshape((recommender.num_users, -1))
permutation = s_filtered.argsort()
rec = item_indices[row, permutation]
# the recommended items will not be exactly determined by
# predicted score; instead, we will sample from the sorted list
# such that higher-preference items get more probability mass
num_items_unseen = rec.shape[1]  # number of items unseen per user
probabilities = np.logspace(0.0, num_items_unseen / 10.0, num=num_items_unseen, base=2)
probabilities = probabilities / probabilities.sum()

# get utility of each item given a state of users
item_states = np.mean(probabilities, axis=0)

# calculate novelty per item by multiplying self information and utility value
item_novelties = items_self_info * item_states
# form sum over all possible items/actions
item_novelty = np.sum(item_novelties)

item_novelty

In [ ]:
def content_fairness(predicted_user_profiles, predicted_item_attributes):
    """
    A score function that ensures that no topic is overrepresented. 
    1. Scores are predicted using the inner product (myopic).
    2. Probabilities are created based on these scores to ensure common value range
    3. The weight of an item to a topic is determined based on its relative value in its embeddings
    4. A slate is created based on myopic scores.
    5. If adding a new item to the slate would exceed the upper bound in a topic dimension, the item is not added.
       Instead, the function proceeds to the next available item until one is found that does not exceed any weights
       in the topic dimensions.
    6. To ensure that the items appear in the top-k slate picked by t-recs, the scores of the slate items are
       manually increased to be in the top-k.
    
    Caveats: Upper_bound and slate_size are set manually because default t-recs implementation does not allow
    passing additional parameters to a score function. The current implementation only works with a top-k slate (non-probabilistic). Also scores are manually altered which reduces interpretability of the scores.
    
    Based on "Controlling Polarization in Personalization: An Algorithmic Framework" by Celis et. al.
    
    Inputs:
        - predicted_user_profiles
        - pretedicted_item_attributes
    Outputs:
        - predicted_scores_reranked (Shape: U x I)
    """
    slate_size = 10
    upper_bound = 0.75

    # 1. Calculate myopic scores
    predicted_scores =  mo.inner_product(predicted_user_profiles, predicted_item_attributes)

    # 2. Calculate probabilities and sort them
    if np.sum(predicted_scores) == 0:
        return predicted_scores
    probs = (predicted_scores.T / np.sum(predicted_scores, axis=1)).T
    probs_sorted = np.flip(np.argsort(probs, axis=1), axis=1)
    
    # 3. Determine weight per embedding dimension for each item (scaled to [0,1])
    gw = predicted_item_attributes.T / np.sum(predicted_item_attributes.T, axis=1)[:, np.newaxis]

    num_user = len(predicted_user_profiles)
    recs = np.empty((num_user, slate_size), dtype=int)
    
    for user in range(len(probs_sorted)):
        agg_weight_per_cluster = np.zeros((len(gw[0]))) # matrix to keep track of weights of slate 
        
        i = 0
        # 4. Create slate in order of myopic scores
        for item in probs_sorted[user]:
            weight_item = gw[item]
            # print(f'{item}: {weight_item}')
            
            # 5. Calculate weights if item was added to slate. If exceeds upper_bound go to next item.
            proposed_weights = weight_item + agg_weight_per_cluster
            if (proposed_weights <= upper_bound).all() and i < slate_size:
                agg_weight_per_cluster = proposed_weights
                recs[user, i] = item
                i += 1

    # 6. Manually assign new scores to items in slate
    predicted_scores_reranked = np.copy(predicted_scores)
    for i, user in enumerate(recs):
        for item in np.flip(user):
            predicted_scores_reranked[int(i), int(item)] = np.max(predicted_scores_reranked[int(i)]) + 1

    return predicted_scores_reranked

In [14]:
score_fn == content_fairness

False